# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import json
import pymongo

In [2]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['youtube']

In [3]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [4]:
df_youtube = df_youtube.drop_duplicates(subset=["video_id"])

In [5]:
with open("./data/US_category_id.json", "r") as f:
    data = json.load(f)

In [6]:
categories = [{"category_id": item["id"], "category_name": item["snippet"]["title"]} for item in data["items"]]

In [7]:
df_categories = pd.DataFrame(categories)
df_categories['category_id'] = df_categories['category_id'].astype(int)

df_youtube_merged = pd.merge(df_youtube, df_categories, on="category_id", how="left")

In [8]:
df_youtube_merged.head(5)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,category_name
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09,Entertainment
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09,Science & Technology
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09,People & Blogs
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09,Science & Technology
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09,Comedy


### Importer les données

In [9]:
collection.delete_many({})
documents = df_youtube.rename(columns={"video_id":"_id"}).to_dict(orient='records')
collection.insert_many(documents)

InsertManyResult(['XpVt6Z1Gjjo', 'K4wEI5zhHB0', 'cLdxuaxaQwc', 'WYYvHb03Eog', 'sjlHnJvXdQs', 'cMKX2tE5Luk', '8wNr-NQImFg', '_HTXMhKWqnA', '_ANP3HR1jsM', 'zgLtEob6X-Q', 'Ayb_2qbZHm4', 'CsdzflTXBVQ', 'l864IBj7cgw', '4MkC65emkG4', 'vu_9muoxT50', '1L7JFN7tQLs', 'ZQK1F0wz6z4', 'T_PuZBdT2iM', 'w8fAellnPns', 'UCrBICYM0yM', '-Ifnaxi2LQg', 'B7YaMkCl3XA', '5ywKal6-anc', '4Yue-q9Jdbk', 'JhA1Wi9mrns', 'EVp4-qjWVJE', 'LcZ2AuvxXNA', 'MdzGZv3zQ-U', '1ZNZY-gd3K0', '2wxyDrfwlXQ', 'QBGaO89cBMI', 'ql0Op1VcELw', 'JO7X9ZPoAp8', 'GGm0FQ6i74U', 'a7Sf_H2cFdM', 'oDIDZ9EmQfA', 'zAtHxJvSczA', 'l_uNfGY3v8E', 'Rew50oYjRqA', 'WWexI9YiLSc', 'LDcm6twPEJA', 'O78Lpo4ctSE', 'qJJHhVf3_ZM', '3yoGE3v4A8w', 'zTjcPeb2Gwg', 'hQbeB0YXLhs', 'L3f7_y9UPh4', 'HwYK5Ay1YNQ', '6vGg-jJl30A', '9CILSvf1snQ', 'WoPtuVbaSKQ', '62cWuFjzOgs', 'NzRuDD0iYC0', 'eM_FR7I2Ttw', 'eoArC99zm1M', 'IMG0u0_cKNc', 'bAon04ZJhHE', 'LTnl36-0-zU', 'EmjMWi0MBXQ', 'c7ymY8QEB2g', 'Ez95rPffd8k', '08RHSZvixec', 'zAXn-v9RdXo', '_SRNenu07Lw', 'iXexmZm3_j8', 'GpAuCG

## Question 1  

In [17]:
cur = collection.find({"channel_title":"Apple"})
list(cur)
#len(list(cur))

[{'_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
  'views': 7860119,
  'likes': 185853,
  'dislikes': 26679,
  'comment_total': 0,
  'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
  'date': 13.09},
 {'_id': 'A9k88sMyiJM',
  'title': 'Apple Watch Series 3 + Apple Music — Roll — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|airpods|apple watch|apple watch series 3|apple watch cellular|apple watch 4g|apple music|40 million songs on your wrist|apple skateboard ad|apple train station ad|apple roll ad|apple watch skateboard ad|apple watch train station ad|apple watch roll ad|airpods skateboard ad|airpods train station ad|airpods roll ad|apple music skateboard ad|apple music train station ad|apple music roll ad|misbehaving',
 

## Question 2

In [18]:
cur = collection.distinct("category_id")
list(cur)

[1, 2, 10, 15, 17, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 43]

## Question 3

In [19]:
collection.update_many({},[{'$set': {'tags': {"$split":["$tags","|"]}}}])

UpdateResult({'n': 2364, 'nModified': 2364, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

## Question 4

In [20]:
cur = collection.find().sort("views", -1).limit(10)
list(cur)

[{'_id': 'tt2k8PGm-TI',
  'title': 'ZAYN - Dusk Till Dawn ft. Sia',
  'channel_title': 'ZaynVEVO',
  'category_id': 10,
  'tags': ['Dusk Till Dawn', 'Pop', 'RCA Records Label', 'ZAYN feat. Sia'],
  'views': 36323498,
  'likes': 1431683,
  'dislikes': 28049,
  'comment_total': 100661,
  'thumbnail_link': 'https://i.ytimg.com/vi/tt2k8PGm-TI/default.jpg',
  'date': 13.09},
 {'_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'],
  'views': 20565649,
  'likes': 1478119,
  'dislikes': 46477,
  'comment_total': 501435,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 19.09},
 {'_id': 'D59v74k5flU',
  'title': 'Primitive Technology: Mud Bricks',
  'channel_title': 'Primitive Technology',
  'category_id': 22,
  'tags': ['[none]'],
  'views': 14068244,
  'likes': 328200,
  'dislikes': 7753,
  'comment_total': 34273,
  'thumbnail_link': 'https

## Question 5

In [21]:
cur = collection.aggregate([{"$group":{"_id":"$category_id", "AverageViews":{"$avg":"$views"}}}])
list(cur)

[{'_id': 1, 'AverageViews': 558289.1881188119},
 {'_id': 19, 'AverageViews': 300664.1111111111},
 {'_id': 17, 'AverageViews': 470686.1677852349},
 {'_id': 25, 'AverageViews': 374883.0505050505},
 {'_id': 23, 'AverageViews': 828485.2676056338},
 {'_id': 29, 'AverageViews': 572023.2},
 {'_id': 26, 'AverageViews': 405748.2900763359},
 {'_id': 28, 'AverageViews': 515854.13907284767},
 {'_id': 10, 'AverageViews': 697598.5910447761},
 {'_id': 2, 'AverageViews': 415129.0540540541},
 {'_id': 20, 'AverageViews': 492390.2413793103},
 {'_id': 27, 'AverageViews': 368229.0425531915},
 {'_id': 15, 'AverageViews': 412381.6216216216},
 {'_id': 24, 'AverageViews': 555071.1740041929},
 {'_id': 43, 'AverageViews': 8259.0},
 {'_id': 22, 'AverageViews': 597891.1789883268}]

## Question 6 

In [22]:
cur = collection.aggregate([{"$group":{"_id":"$channel_title", "Averagelikes":{"$avg":"$likes"}}}, {"$sort": {"Averagelikes":-1}}])
list(cur)

[{'_id': 'ZaynVEVO', 'Averagelikes': 1431683.0},
 {'_id': 'ibighit', 'Averagelikes': 1112071.0},
 {'_id': 'melanie martinez', 'Averagelikes': 902166.0},
 {'_id': 'jypentertainment', 'Averagelikes': 721796.0},
 {'_id': 'Logan Paul Vlogs', 'Averagelikes': 601533.5},
 {'_id': 'Wengie', 'Averagelikes': 455904.0},
 {'_id': 'Conor Maynard', 'Averagelikes': 389273.0},
 {'_id': 'Dude Perfect', 'Averagelikes': 363320.0},
 {'_id': 'PewDiePie', 'Averagelikes': 351994.0},
 {'_id': 'Primitive Technology', 'Averagelikes': 328200.0},
 {'_id': 'Gabbie Hanna', 'Averagelikes': 315097.0},
 {'_id': 'Alesso', 'Averagelikes': 306843.0},
 {'_id': 'Ed Sheeran', 'Averagelikes': 306217.0},
 {'_id': 'Liza Koshy', 'Averagelikes': 306043.2},
 {'_id': 'FifthHarmonyVEVO', 'Averagelikes': 291383.0},
 {'_id': 'Daniel Howell', 'Averagelikes': 271091.0},
 {'_id': 'ImagineDragonsVEVO', 'Averagelikes': 259123.0},
 {'_id': 'Bhad Bhabie', 'Averagelikes': 236196.0},
 {'_id': 'Linkin Park', 'Averagelikes': 231182.0},
 {'_id':